In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plot import *
from dynamic_stop_loss import *
from cost_benchmark import cost_benchmark
import calendar


# Set parameters
np.random.seed(42)

year = 2024
days = 366 if calendar.isleap(year) else 365
simulations = 1000  # Number of simulations
initial_price = 50  # Initial price in EUR/MWh
mu = 0.0005  # Drift (expected daily return)
sigma = 0.005  # Volatility (daily standard deviation of returns)

# Monte Carlo simulation for multiple price paths
def monte_carlo_simulation(initial_price, mu, sigma, days, simulations):
    price_matrix = np.zeros((simulations, days))
    price_matrix[:, 0] = initial_price

    for t in range(1, days):
        # Generate daily returns based on normal distribution (drift + volatility)
        random_returns = np.random.normal(mu, sigma, simulations)
        price_matrix[:, t] = price_matrix[:, t - 1] * np.exp(random_returns)
    
    return price_matrix

# Run the simulation
price_paths = monte_carlo_simulation(initial_price, mu, sigma, days, simulations)

# Create a DataFrame for plotting
dates = pd.date_range(start='2024-01-01', periods=days)
df_simulations = pd.DataFrame(price_paths.T, index=dates)

# Plot a few random simulations (for visualization purposes)
plt.figure(figsize=(20, 6))
for i in range(simulations):  # Plot only 10 random simulations for clarity
    plt.plot(df_simulations.index, df_simulations.iloc[:, i], lw=1)
plt.title(f'Monte Carlo Simulated Electricity Price Paths ({simulations} Simulations)')
plt.xlabel('Date')
plt.ylabel('Price (EUR/MWh)')
plt.grid(True)
plt.show()

In [2]:
spread = 0.8

In [ ]:
all_saving_stats = []
for i in range(simulations):
    df_single = pd.DataFrame({
        "Price": df_simulations.iloc[:, i]
    })
    df_single.index.name = "Dates"  
    df_single.reset_index(inplace=True)
    
    procurement = quarterly_procurement(df_single, spread)
    average_price, average_procurement_price, total_average_cost, total_procurement_cost, savings = cost_benchmark("single simulation", df_single, procurement)
    
    all_saving_stats.append(savings)

# 计算所有 savings 的平均值
average_savings = np.mean(all_saving_stats)
count_positive = sum(1 for s in all_saving_stats if s > 0)
print(f"Average Savings: {average_savings}")
print(f"Number of positive savings: {count_positive}")